In [ ]:
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from multiprocessing import Pool
import os

In [ ]:
def progress(value, max=100):
    return HTML(
        """
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(
            value=value, max=max
        )
    )


def get_summary_data(soup):
    # Summary data is not in a nice table, so we have to pick it out from the divs and list items.
    case_title = soup.select_one("span.caseHdrTitle").text.strip()
    case_id = case_title[:15]
    party = soup.findAll("div", {"class": "ptyInfoLabel"})[0].text.strip()
    party_type = soup.findAll("div", {"class": "ptyType"})[0].text.strip()
    try:
        party_aka = soup.findAll("li", {"class": "ptyAfflName"})[0].text.strip()
    except:
        party_aka = ""
    try:
        party_attny = soup.findAll("li", {"class": "ptyAttyInfo"})[0].text.strip()
    except:
        party_attny = ""
    data = [
        ["case_id", case_id],
        ["case_title", case_title],
        ["party", party],
        ["party_type", party_type],
        ["party_aka", party_aka],
        ["party_attny", party_attny],
    ]
    case_info = soup.find(
        lambda tag: tag.name == "div"
        and tag.has_attr("id")
        and tag["id"] == "caseHeader"
    )
    rows = case_info.find_all("ul")
    for row in rows:
        cols = row.find_all("li")
        col = [x.text.strip() for x in cols]
        data.append(col)
    summary = pd.DataFrame(
        columns=[
            "action",
            "case_status",
            "case_type",
            "file_date",
            "next_event",
            "status_date",
            "case_id",
            "case_title",
            "party",
            "party_aka",
            "party_attny",
            "party_type",
        ]
    )
    summary = pd.concat([summary, pd.DataFrame(pd.Series(dict(data))).T], sort=False)
    summary = summary[
        [
            "case_id",
            "case_title",
            "party",
            "party_type",
            "party_aka",
            "party_attny",
            "file_date",
            "case_type",
            "action",
            "status_date",
            "case_status",
            "next_event",
        ]
    ]
    return summary


def get_receipts(soup, case_id):
    if len(
        soup.findAll(
            "table",
            {
                "class": "tablesorter",
                "summary": "Table displaying the Receipts on this case.",
            },
        )
    ):
        receipts = pd.read_html(
            str(
                soup.findAll(
                    "table",
                    {
                        "class": "tablesorter",
                        "summary": "Table displaying the Receipts on this case.",
                    },
                )
            ),
            header=0,
        )[0]
        receipts.columns = [
            "Receipt Number",
            "Receipt Date",
            "Received From",
            "Payment Amount",
        ]
        receipts["case_id"] = case_id
        receipts = receipts[: len(receipts) - 1]  # drop empty totals
    else:
        receipts = pd.DataFrame()
    return receipts


def get_events(soup, case_id):
    events = pd.read_html(
        str(soup.findAll("table", {"class": "tablesorter"})[0]), header=0
    )[0]
    try:
        events.columns = ["DateTime", "Location", "Type", "Result", "Judge"]
    except:
        print(case_id, events.columns)
    events["case_id"] = case_id
    return events


def get_docket(soup, case_id):
    docket = pd.read_html(
        str(soup.findAll("table", {"class": "tablesorter"})[1]), header=0
    )[0]
    try:
        docket.columns = ["Date", "Docket Text", "Image Available"]
    except:
        print(case_id, docket.columns)
    docket["case_id"] = case_id
    return docket


def get_disposition(soup, case_id):
    disp = soup.findAll(
        "table",
        {
            "class": "tablesorter",
            "summary": "Table displaying the dispositions on this case ordered chronologically.",
        },
    )
    if len(disp):
        disposition = pd.read_html(str(disp[0]), header=0)[0]
        try:
            disposition.columns = ["Disposition", "Date", "Case Judge"]
        except:
            print(case_id, disposition.columns)
        disposition["case_id"] = case_id
    else:
        disposition = pd.DataFrame()
    return disposition


def get_data(soup):
    summary = get_summary_data(soup)
    case_id = summary["case_id"][0]
    events = get_events(soup, case_id)
    docket = get_docket(soup, case_id)
    receipts = get_receipts(soup, case_id)
    docket = get_docket(soup, case_id)
    disposition = get_disposition(soup, case_id)
    return summary, events, docket, receipts, disposition


def get_data_for_one_year(base_path, year):
    summary_df = pd.DataFrame()
    events_df = pd.DataFrame()
    docket_df = pd.DataFrame()
    receipts_df = pd.DataFrame()
    dispositions_df = pd.DataFrame()
    scrape_path = Path(base_path.parent, "DCSC-Scraper", str(year))
    list_of_paths = sorted(scrape_path.rglob("*.html"))
    print("There are", len(list_of_paths), "scraped cases in", year)
    completed = 0
    total = len(list_of_paths)
    out = display(progress(0, 100), display_id=True)
    for path in list_of_paths:
        soup = BeautifulSoup(open(path), "html.parser")
        summary, events, docket, receipts, disposition = get_data(soup)
        summary_df = pd.concat([summary_df, summary], sort=False)
        events_df = pd.concat([events_df, events], sort=False)
        docket_df = pd.concat([docket_df, docket], sort=False)
        receipts_df = pd.concat([receipts_df, receipts], sort=False)
        dispositions_df = pd.concat([dispositions_df, disposition], sort=False)
        completed += 1
        percent_done = round((completed / total) * 100)
        out.update(progress(percent_done, 100))
    return summary_df, events_df, docket_df, receipts_df, dispositions_df

In [ ]:
base_path = Path.cwd()
def parse_cases(year):
    summary_df, events_df, docket_df, receipts_df, dispositions_df = get_data_for_one_year(base_path, year)
    summary_df.to_csv('summary_data_'+str(year)+'.csv', index=False)
    events_df.to_csv('events_data_'+str(year)+'.csv', index=False)
    docket_df.to_csv('docket_data_'+str(year)+'.csv', index=False)
    receipts_df.to_csv('receipts_data_'+str(year)+'.csv', index=False)
    dispositions_df.to_csv('dispositions_data_'+str(year)+'.csv', index=False)


In [ ]:
years = list(range(2014,2020))

if __name__ == '__main__':
    pool = Pool(os.cpu_count()-1)              # Create a multiprocessing Pool
    pool.map(parse_cases, years)  # process iterable with pool


In [ ]:
print('Done!')